In [2]:
# === setup_bookdown.R ===

# 📁 設定章節來源目錄（從當前資料夾開始遞迴找所有 .md / .Rmd）
library(stringr)

# --- STEP 1: 修復中間誤用的 ---
replace_unsafe_hr <- function(file) {
  lines <- readLines(file, warn = FALSE)
  yaml_start <- which(lines == "---")[1]
  yaml_end   <- which(lines == "---")[2]
  
  # 沒有 YAML 區塊，直接處理整篇
  if (is.na(yaml_start) || is.na(yaml_end)) {
    lines_fixed <- gsub("^---$", "***", lines)
  } else {
    # 修正 YAML 區塊外的 --- 為 ***
    lines_fixed <- c(
      lines[1:yaml_end],
      gsub("^---$", "***", lines[(yaml_end + 1):length(lines)])
    )
  }
  
  writeLines(lines_fixed, file)
  message("✅ 已修正 `---` 為 `***` ：", file)
}


In [3]:

# --- STEP 2: 掃描並修正所有 .md 檔案 ---
all_md <- list.files(".", pattern = "\\.md$", recursive = TRUE, full.names = TRUE)
invisible(lapply(all_md, replace_unsafe_hr))



<U+2705> <U+5DF2><U+4FEE><U+6B63> `---` <U+70BA> `***` <U+FF1A>./<e8><aa><b2><e5><a0><82><e7><ad><86><e8><a8><98>/00_Weekly/W01_0218.md

<U+2705> <U+5DF2><U+4FEE><U+6B63> `---` <U+70BA> `***` <U+FF1A>./<e8><aa><b2><e5><a0><82><e7><ad><86><e8><a8><98>/00_Weekly/W05_0318.md

<U+2705> <U+5DF2><U+4FEE><U+6B63> `---` <U+70BA> `***` <U+FF1A>./<e8><aa><b2><e5><a0><82><e7><ad><86><e8><a8><98>/01_Staatsrecht<e9><81><b8><e8><ae><80>/Rn_338.md

<U+2705> <U+5DF2><U+4FEE><U+6B63> `---` <U+70BA> `***` <U+FF1A>./<e8><aa><b2><e5><a0><82><e7><ad><86><e8><a8><98>/01_Staatsrecht<e9><81><b8><e8><ae><80>/Rn_339.md

<U+2705> <U+5DF2><U+4FEE><U+6B63> `---` <U+70BA> `***` <U+FF1A>./<e8><aa><b2><e5><a0><82><e7><ad><86><e8><a8><98>/01_Staatsrecht<e9><81><b8><e8><ae><80>/Rn_340.md

<U+2705> <U+5DF2><U+4FEE><U+6B63> `---` <U+70BA> `***` <U+FF1A>./<e8><aa><b2><e5><a0><82><e7><ad><86><e8><a8><98>/01_Staatsrecht<e9><81><b8><e8><ae><80>/Rn_341.md

<U+2705> <U+5DF2><U+4FEE><U+6B63> `---` <U+70BA> `***` <U+FF1A>./<e8><

In [4]:


# --- STEP 3: 建立 _bookdown.yml ---
all_files <- list.files(".", pattern = "\\.R?md$", recursive = TRUE, full.names = TRUE)
# 排除 _bookdown.yml 本身、README、開頭是 "_" 的檔案
valid_files <- all_files[!grepl("^_", basename(all_files))]
valid_files <- valid_files[!basename(valid_files) %in% c("README.md", "README.Rmd")]
valid_files <- sort(valid_files)
# 將 index.Rmd 放最前面
valid_files <- c("index.Rmd", setdiff(valid_files, "index.Rmd"))

# YAML 內容
yaml_lines <- c(
  'book_filename: "1132-Readings-of-German-Jurisprudence.Rmd"',
  'clean: [packages.bib, bookdown.bbl]',
  'delete_merged_file: true',
  'language:',
  '  label:',
  '    fig: "圖 "',
  '    tab: "表 "',
  '  ui:',
  '    edit: "編輯"',
  '    chapter_name: ["第 ", " 章"]',
  'output_dir: "docs"',
  'rmd_subdir: true',
  'rmd_files:',
  paste0("  - ", valid_files)
)

writeLines(yaml_lines, "_bookdown.yml")
cat("\n✅ _bookdown.yml 產生成功，共 ", length(valid_files), " 個章節！\n")

# --- STEP 4: 選擇性立即渲染書籍 ---
# bookdown::render_book("index.Rmd", "bookdown::gitbook")


<U+2705> _bookdown.yml <U+7522><U+751F><U+6210><U+529F><U+FF0C><U+5171>  20  <U+500B><U+7AE0><U+7BC0><U+FF01>


In [5]:
files <- list.files(".", pattern = "\\.R?md$", recursive = TRUE, full.names = TRUE)
for (f in files) {
  lines <- readLines(f, warn = FALSE)
  if (!any(grepl("^# ", lines))) {
    message("⚠️ 檔案缺少章節標題（# 開頭）➡ ", f)
  }
}

In [1]:
# === add_anchor_to_headers.R ===
library(stringi)

# 轉換中文標題 ➜ 合法 GitBook ID slug
slugify <- function(text) {
  text <- stringi::stri_trans_general(text, "Latin-ASCII") # 轉換變成類拼音/英文
  text <- tolower(text)
  text <- gsub("[^a-z0-9]+", "-", text)       # 非字母數字轉成 -
  text <- gsub("(^-|-$)", "", text)           # 去頭尾的 -
  return(text)
}

# 為章節標題補上 {#slug}
add_anchors <- function(file) {
  lines <- readLines(file, warn = FALSE)
  modified <- FALSE
  
  lines <- lapply(lines, function(line) {
    if (grepl("^# ", line) && !grepl("\\{#.*\\}", line)) {
      title <- gsub("^# +", "", line)
      title_clean <- gsub("\\*+", "", title)                  # 去除粗體符號
      title_clean <- gsub("[（(].*?[）)]", "", title_clean)   # 去除括號中的說明
      slug <- slugify(title_clean)
      line <- paste0(line, " {#", slug, "}")
      modified <<- TRUE
    }
    return(line)
  })
  
  if (modified) {
    writeLines(unlist(lines), file)
    message("✅ 已補 anchor：", file)
  } else {
    message("☑️ 已有 anchor 或無標題：", file)
  }
}

# 處理所有 .md / .Rmd
files <- list.files(".", pattern = "\\.R?md$", recursive = TRUE, full.names = TRUE)
lapply(files, add_anchors)

<U+2705> <U+5DF2><U+88DC> anchor<U+FF1A>./<e8><aa><b2><e5><a0><82><e7><ad><86><e8><a8><98>/00_Weekly/W01_0218.md

<U+2705> <U+5DF2><U+88DC> anchor<U+FF1A>./<e8><aa><b2><e5><a0><82><e7><ad><86><e8><a8><98>/00_Weekly/W05_0318.md

<U+2705> <U+5DF2><U+88DC> anchor<U+FF1A>./<e8><aa><b2><e5><a0><82><e7><ad><86><e8><a8><98>/01_Staatsrecht<e9><81><b8><e8><ae><80>/Rn_338.md

<U+2705> <U+5DF2><U+88DC> anchor<U+FF1A>./<e8><aa><b2><e5><a0><82><e7><ad><86><e8><a8><98>/01_Staatsrecht<e9><81><b8><e8><ae><80>/Rn_339.md

<U+2705> <U+5DF2><U+88DC> anchor<U+FF1A>./<e8><aa><b2><e5><a0><82><e7><ad><86><e8><a8><98>/01_Staatsrecht<e9><81><b8><e8><ae><80>/Rn_340.md

<U+2705> <U+5DF2><U+88DC> anchor<U+FF1A>./<e8><aa><b2><e5><a0><82><e7><ad><86><e8><a8><98>/01_Staatsrecht<e9><81><b8><e8><ae><80>/Rn_341.md

<U+2705> <U+5DF2><U+88DC> anchor<U+FF1A>./<e8><aa><b2><e5><a0><82><e7><ad><86><e8><a8><98>/01_Staatsrecht<e9><81><b8><e8><ae><80>/Rn_342.md

<U+2705> <U+5DF2><U+88DC> anchor<U+FF1A>./<e8><aa><b2><e5><a0><82><

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL